# Calculate the percent of datapoints dropped by manufacturer

In [1]:
import os
from os.path import join

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np
import pandas as pd

In [3]:
analysisDir = '../analysis/'

In [4]:
df = pd.read_table(join(analysisDir, 'allSubjs_calibrationSummary.tsv'), sep='\t')

In [5]:
df[:10]

,trial,ptIdx,percentValid,centX,centY,centDist,centAngle,RMS,subj,glasses,dist,offset,condition
0,1,15,0.992,872.513,456.424,0.683,48.041,0.113,101,Pupil Labs,1M,0deg,101_PupilLabs_1M_0deg
1,2,21,0.996,178.414,736.665,1.135,83.071,0.074,101,Pupil Labs,1M,0deg,101_PupilLabs_1M_0deg
2,3,3,0.992,529.483,128.252,0.564,52.494,0.114,101,Pupil Labs,1M,0deg,101_PupilLabs_1M_0deg
3,4,13,0.988,537.017,401.570,1.226,69.390,0.123,101,Pupil Labs,1M,0deg,101_PupilLabs_1M_0deg
4,5,25,0.992,862.071,709.513,1.481,76.933,0.087,101,Pupil Labs,1M,0deg,101_PupilLabs_1M_0deg
5,6,5,0.992,871.807,74.899,1.160,67.254,0.087,101,Pupil Labs,1M,0deg,101_PupilLabs_1M_0deg
6,7,1,0.992,183.395,78.256,1.049,79.286,0.085,101,Pupil Labs,1M,0deg,101_PupilLabs_1M_0deg
7,8,23,0.996,500.611,682.510,1.758,89.768,0.080,101,Pupil Labs,1M,0deg,101_PupilLabs_1M_0deg
8,9,11,0.992,146.555,442.042,0.715,109.137,0.203,101,Pupil Labs,1M,0deg,101_PupilLabs_1M_0deg
9,1,25,0.992,838.920,755.500,0.909,85.894,0.152,101,Pupil Labs,1M,10Ldeg,101_PupilLabs_1M_10Ldeg


# Run model
Run Linear Mixed Effects model to test whether percentValid differs as a function of glasses, controlling for subj

In [22]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

### Run the initial model
This will set one of the levels of the 'glasses' var to the intercept, and make comparisons using the other levels

In [23]:
md = smf.mixedlm("percentValid ~ glasses", df, groups=df['subj']).fit()
print(mdf.summary())

           Mixed Linear Model Regression Results
Model:             MixedLM Dependent Variable: percentValid
No. Observations:  729     Method:             REML        
No. Groups:        3       Scale:              0.0019      
Min. group size:   243     Likelihood:         1236.2956   
Max. group size:   243     Converged:          Yes         
Mean group size:   243.0                                   
-----------------------------------------------------------
                 Coef. Std.Err.    z    P>|z| [0.025 0.975]
-----------------------------------------------------------
Intercept        0.971    0.009 110.361 0.000  0.954  0.988
glasses[T.SMI]   0.016    0.004   4.016 0.000  0.008  0.024
glasses[T.Tobii] 0.011    0.004   2.837 0.005  0.003  0.019
groups RE        0.000    0.005                            



/usr/local/lib/python3.6/site-packages/statsmodels/regression/mixed_linear_model.py:2019: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


#### Results:

* Pupil Labs mean: .971 (SE: .009) 
* SMI mean: .987 (SE: .004)
* Tobii mean: .982 (SE: .004)

* Pupil Labs > SMI: z = 4.016, p < .001
* Pupil Labs > Tobii: z = 2.837, p < .001

However, this model doesn't give us the SMI > Tobii test. To get that we run the same model, only now explicitly set
SMI to be the intercept

In [24]:
md2 = smf.mixedlm("percentValid ~ C(glasses, Treatment(reference='SMI'))", df, groups=df['subj']).fit()
print(md2.summary())

                             Mixed Linear Model Regression Results
Model:                          MixedLM             Dependent Variable:             percentValid
No. Observations:               729                 Method:                         REML        
No. Groups:                     3                   Scale:                          0.0019      
Min. group size:                243                 Likelihood:                     1236.2956   
Max. group size:                243                 Converged:                      Yes         
Mean group size:                243.0                                                           
------------------------------------------------------------------------------------------------
                                                     Coef.  Std.Err.    z    P>|z| [0.025 0.975]
------------------------------------------------------------------------------------------------
Intercept                                             0.987 

/usr/local/lib/python3.6/site-packages/statsmodels/regression/mixed_linear_model.py:2019: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


#### Results:

* SMI > Tobii: z = -1.18, p = .238

# Summary

There was a significant mean difference between the glasses manufacturers. Specifically, Pupil Labs had significantly more dropped data points compared to either Tobii or SMI. There was no difference between the Tobii and SMI glasses